# 1. 전처리

In [1]:
import pandas as pd
import numpy as np

In [ ]:
ppl=pd.read_excel('/content/drive/MyDrive/khuda_toy/41_ppl.xlsx')
time=pd.read_csv('/content/drive/MyDrive/khuda_toy/41_inter_time.csv',encoding='cp949')

In [ ]:
ppl.head()

,지하철역 / 총 승하차인원,04시-05시,05시-06시,06시-07시,07시-08시,08시-09시,9시-10시,10시-11시,11시-12시,12시-13시,...,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,00시-01시,01시-02시,02시-03시,03시-04시
0,가산디지털단지,34.060109,641.844262,1943.478142,7748.423498,19888.021860,11326.789620,3869.740437,2893.784153,3135.893443,...,11652.133880,16438.729510,7074.308744,4061.409836,3188.396175,1974.357924,222.658470,0.245902,0.010929,0
1,강남구청,1.079235,174.153006,1001.396175,2062.721311,5568.887980,5438.196723,2511.877049,1765.136612,1739.718579,...,3415.385246,5340.674862,3272.032787,2059.057377,1902.560109,1399.418033,170.767760,0.193989,0.000000,0
2,건대입구,3.032787,264.833333,656.642076,1610.133880,2858.767760,2248.311476,2005.663934,1915.868852,2091.647541,...,3549.871585,3803.765027,2594.718579,2139.008197,1820.090164,1594.002732,262.106557,11.281421,0.103825,0
3,고속터미널,1.923497,193.961749,1236.445355,2398.866120,3394.505464,3042.765028,2711.789617,2731.713115,2645.442623,...,3970.707650,3820.972678,2515.759563,2290.636612,2072.926230,1525.909836,74.994536,0.478142,0.000000,0
4,공릉,0.953552,362.142076,920.685792,2312.459016,3222.423497,2065.964481,1568.836066,1490.964481,1693.057377,...,2502.128415,3176.568306,2446.027322,1567.278689,1431.931694,1335.907104,252.385246,0.420765,0.000000,0


In [ ]:
time.head()

,역명,소요시간,역간거리(km),누적 역간거리(km),grade
0,도봉산,0,1.4,1.4,c
1,수락산,140,1.6,3.0,c
2,마들,120,1.4,4.4,c
3,노원,90,1.2,5.6,b
4,중계,80,1.1,6.7,c


## 1-1. 실제 운행 순서로 역 순서 정렬

In [ ]:
# 공통된 기준이 'id'라는 컬럼일 경우
time_idx = time.set_index('역명')   # time의 인덱스를 id로 설정
ppl_idx = ppl.set_index('지하철역 / 총 승하차인원')     # ppl의 인덱스를 id로 설정

# time의 인덱스 순서에 맞게 ppl 정렬
ppl_sorted = ppl_idx.loc[time_idx.index]


## 1-2. 통근시간 승하차인원 평균

In [ ]:
commute_morning = ((ppl_sorted['07시-08시'] + ppl_sorted['08시-09시']+ppl_sorted['9시-10시'])/3).tolist() # 07-10
commute_night = ((ppl_sorted['17시-18시'] + ppl_sorted['18시-19시']+ppl_sorted['19시-20시'])/3).tolist() # 17-20
inter_time = (time['소요시간']).tolist()
grade=(time['grade']).tolist()


In [ ]:
commute_avg = [(m + n) / 2 for m, n in zip(commute_morning, commute_night)]

In [ ]:
commute_avg[3]

4509.840619

In [ ]:
sum(commute_avg)

135794.20010688328

In [ ]:
len(commute_avg)

41

In [ ]:
commute_avg

[2260.2959926666663,
 2681.989982,
 2812.380236666667,
 4509.840619,
 3199.374317,
 3807.5719493333336,
 2620.928506166667,
 1896.3907101666664,
 2267.2718576666666,
 2407.0141164999995,
 3898.1338795,
 3415.3861566666665,
 3641.031876,
 1336.0227686666667,
 2487.3907103333336,
 3096.838798,
 3339.249544666666,
 2777.5947178333336,
 2012.1996488333334,
 5084.098816666667,
 4182.9831515000005,
 6325.1507280000005,
 4443.2818765,
 1255.4877050500002,
 3190.5960838333335,
 3559.604736,
 3423.144808833333,
 2504.2299635,
 2955.874772166667,
 2550.4503645,
 1979.3201273333334,
 3331.1853371666666,
 1615.5040984999998,
 2487.9845173333333,
 1752.5997268333335,
 3966.9385250000005,
 12354.734518666668,
 4918.889799833332,
 4451.4649365000005,
 2671.3155738333335,
 2322.4535516666665]

# New 가중치 부여한경우

In [ ]:
def find_maximum_subarray(A, k, T_max):
    # 배열 A의 길이
    n = len(A)

    # 최대 합과 구간의 인덱스 초기화
    max_sum = 0
    start_index = 0
    end_index = 0

    # 초기 슬라이딩 윈도우 설정 (길이 k-1)
    current_sum = sum(A[:k-1])

    # 처음 구간이 최대 합 이하이면 설정
    if current_sum <= T_max:
        max_sum = current_sum
        end_index = k-2

    # 슬라이딩 윈도우로 k-1 크기의 부분 구간을 이동하면서 최대 합 찾기
    for i in range(k-1, n):
        # 슬라이딩 윈도우 오른쪽으로 이동
        current_sum += A[i] - A[i-(k-1)]

        # 합이 T_max를 넘지 않으면 최대 합 갱신
        if current_sum <= T_max and current_sum > max_sum:
            max_sum = current_sum
            start_index = i - (k-2)
            end_index = i

    return max_sum, start_index, end_index

def maximize_sum_with_constraints(P, start_index, end_index):
    # P 리스트의 길이
    n = len(P)

    # start_index와 end_index는 선택된 상태에서, 나머지 15개의 인덱스를 선택해야 함
    selected_indices = {start_index, end_index}

    # start_index와 end_index 사이의 구간에서 가능한 인덱스들만 고려
    candidate_indices = list(range(start_index + 1, end_index))

    # candidate_indices 중 P의 값을 기준으로 정렬
    candidate_indices.sort(key=lambda x: P[x], reverse=True)

    # 최대 합을 계산하기 위해 큰 값의 인덱스를 선택
    for idx in candidate_indices:
        if len(selected_indices) < 17:
            selected_indices.add(idx)
        else:
            break

    # 선택한 인덱스들의 합 계산
    max_sum = sum(P[i] for i in selected_indices)
d
    # 선택된 인덱스들을 리스트로 정렬하여 반환
    selected_indices = sorted(list(selected_indices))

    return max_sum, selected_indices

# 예제 사용
A = inter_time
P = commute_avg
grade_weights = {'a': 1 + 0.649, 'b': 1 + 0.232, 'c': 1 + 0.119}

# 2. 가중치 적용
weighted_P = [P[i] * grade_weights[station_grades[i]] for i in range(n)]

# 각 k에 대해 최대 합과 구간을 찾고 최적화된 선택을 진행
for k in range(17, 33):
    T_max = 3290 - 16 * 30  # 예제 값, 필요에 따라 수정
    subarray_result = find_maximum_subarray(A, k, T_max)
    start_index, end_index = subarray_result[1], subarray_result[2]

    # P 리스트에서 최대화된 합을 구하는 최적화
    optimization_result = maximize_sum_with_constraints(weighted_P, start_index, end_index)
    result = find_maximum_subarray(A, k, T_max)

    print(f"k={k}, 최대 합: {sum(P[i] for i in optimization_result[1])}, 선택된 인덱스: {optimization_result[1]}")
    print(time.loc[optimization_result[1], '역명'])
    print(f'총소요시간:{result[0]-A[start_index]+16*30}, {result[1]}, {result[2]}')

k=17, 최대 합: 56845.69535766667, 선택된 인덱스: [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
25         내방
26         이수
27         남성
28      숭실대입구
29         상도
30       장승배기
31     신대방삼거리
32        보라매
33         신풍
34         대림
35        남구로
36    가산디지털단지
37         철산
38      광명사거리
39         천왕
40         온수
Name: 역명, dtype: object
총소요시간:2120, 25, 40
k=18, 최대 합: 60036.2914415, 선택된 인덱스: [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
24      고속터미널
25         내방
26         이수
27         남성
28      숭실대입구
29         상도
30       장승배기
31     신대방삼거리
32        보라매
33         신풍
34         대림
35        남구로
36    가산디지털단지
37         철산
38      광명사거리
39         천왕
40         온수
Name: 역명, dtype: object
총소요시간:2270, 24, 40
k=19, 최대 합: 59676.275048049996, 선택된 인덱스: [23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40]
23         반포
24      고속터미널
25         내방
26         이수
27         남성
28      숭실대입구
29         상도
30       장승배기
31     신대방삼거리
33    

# New 가중치 없는경우

In [ ]:
def find_maximum_subarray(A, k, T_max):
    # 배열 A의 길이
    n = len(A)

    # 최대 합과 구간의 인덱스 초기화
    max_sum = 0
    start_index = 0
    end_index = 0

    # 초기 슬라이딩 윈도우 설정 (길이 k-1)
    current_sum = sum(A[:k-1])

    # 처음 구간이 최대 합 이하이면 설정
    if current_sum <= T_max:
        max_sum = current_sum
        end_index = k-2

    # 슬라이딩 윈도우로 k-1 크기의 부분 구간을 이동하면서 최대 합 찾기
    for i in range(k-1, n):
        # 슬라이딩 윈도우 오른쪽으로 이동
        current_sum += A[i] - A[i-(k-1)]

        # 합이 T_max를 넘지 않으면 최대 합 갱신
        if current_sum <= T_max and current_sum > max_sum:
            max_sum = current_sum
            start_index = i - (k-2)
            end_index = i

    return max_sum, start_index, end_index

def maximize_sum_with_constraints(P, start_index, end_index):
    # P 리스트의 길이
    n = len(P)

    # start_index와 end_index는 선택된 상태에서, 나머지 15개의 인덱스를 선택해야 함
    selected_indices = {start_index, end_index}

    # start_index와 end_index 사이의 구간에서 가능한 인덱스들만 고려
    candidate_indices = list(range(start_index + 1, end_index))

    # candidate_indices 중 P의 값을 기준으로 정렬
    candidate_indices.sort(key=lambda x: P[x], reverse=True)

    # 최대 합을 계산하기 위해 큰 값의 인덱스를 선택
    for idx in candidate_indices:
        if len(selected_indices) < 17:
            selected_indices.add(idx)
        else:
            break

    # 선택한 인덱스들의 합 계산
    max_sum = sum(P[i] for i in selected_indices)

    # 선택된 인덱스들을 리스트로 정렬하여 반환
    selected_indices = sorted(list(selected_indices))

    return max_sum, selected_indices

# 예제 사용
A = inter_time
P = commute_avg

# 각 k에 대해 최대 합과 구간을 찾고 최적화된 선택을 진행
for k in range(17, 33):
    T_max = 3290 - 16 * 30  # 예제 값, 필요에 따라 수정
    subarray_result = find_maximum_subarray(A, k, T_max)
    start_index, end_index = subarray_result[1], subarray_result[2]

    # P 리스트에서 최대화된 합을 구하는 최적화
    optimization_result = maximize_sum_with_constraints(P, start_index, end_index)
    result = find_maximum_subarray(A, k, T_max)

    print(f"k={k}, 최대 합: {sum(P[i] for i in optimization_result[1])}, 선택된 인덱스: {optimization_result[1]}")
    print(time.loc[optimization_result[1], '역명'])
    print(f'총소요시간:{result[0]-A[start_index]+16*30}, {result[1]}, {result[2]}')

k=17, 최대 합: 56845.69535766667, 선택된 인덱스: [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
25         내방
26         이수
27         남성
28      숭실대입구
29         상도
30       장승배기
31     신대방삼거리
32        보라매
33         신풍
34         대림
35        남구로
36    가산디지털단지
37         철산
38      광명사거리
39         천왕
40         온수
Name: 역명, dtype: object
총소요시간:2120, 25, 40
k=18, 최대 합: 60036.2914415, 선택된 인덱스: [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
24      고속터미널
25         내방
26         이수
27         남성
28      숭실대입구
29         상도
30       장승배기
31     신대방삼거리
32        보라매
33         신풍
34         대림
35        남구로
36    가산디지털단지
37         철산
38      광명사거리
39         천왕
40         온수
Name: 역명, dtype: object
총소요시간:2270, 24, 40
k=19, 최대 합: 59676.275048049996, 선택된 인덱스: [23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40]
23         반포
24      고속터미널
25         내방
26         이수
27         남성
28      숭실대입구
29         상도
30       장승배기
31     신대방삼거리
33    